In [1]:
import pandas as pd 
df = pd.read_csv("unicode-char-categories.csv")
# From https://www.fileformat.info/info/unicode/category/index.htm

In [2]:
from transformers import AutoTokenizer

/Users/shannon/anaconda3/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("allenai/ivila-row-layoutlm-finetuned-s2vl-v2")

I0928 13:51:53.949661 4404223424 filelock.py:274] Lock 140539838121576 acquired on /Users/shannon/.cache/huggingface/transformers/b723c27ef9c2c0dcb14e2579b1c4813399147edc91358c588657f48079bd37c4.0ee3de34eb96674c87647d1bc7197ef5dca8f7ed09679670c5e4fa17a1dce45a.lock


Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

I0928 13:51:54.105487 4404223424 filelock.py:318] Lock 140539838121576 released on /Users/shannon/.cache/huggingface/transformers/b723c27ef9c2c0dcb14e2579b1c4813399147edc91358c588657f48079bd37c4.0ee3de34eb96674c87647d1bc7197ef5dca8f7ed09679670c5e4fa17a1dce45a.lock
I0928 13:51:54.222512 4404223424 filelock.py:274] Lock 140539940190248 acquired on /Users/shannon/.cache/huggingface/transformers/12219894b117ccf78f257f5361cde58cdfabfebc79fee721f8cb0504139179f9.a5784bb4d9b1c181841fc2293cd92bd2778b2284fcc5bd9a91a12c5e5f11fab0.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

I0928 13:51:54.433845 4404223424 filelock.py:318] Lock 140539940190248 released on /Users/shannon/.cache/huggingface/transformers/12219894b117ccf78f257f5361cde58cdfabfebc79fee721f8cb0504139179f9.a5784bb4d9b1c181841fc2293cd92bd2778b2284fcc5bd9a91a12c5e5f11fab0.lock
I0928 13:51:54.538693 4404223424 filelock.py:274] Lock 140539940190920 acquired on /Users/shannon/.cache/huggingface/transformers/8b3025e00d8d7a8f3857e0d178c15888924b302167053c7e48af38f837baf20d.906feaed289524cc12e69304a4700cfc05d1f19a278001ef8ec7d3f4c7773c84.lock


...

I0928 13:51:54.829241 4404223424 filelock.py:318] Lock 140539940190920 released on /Users/shannon/.cache/huggingface/transformers/8b3025e00d8d7a8f3857e0d178c15888924b302167053c7e48af38f837baf20d.906feaed289524cc12e69304a4700cfc05d1f19a278001ef8ec7d3f4c7773c84.lock
I0928 13:51:55.047997 4404223424 filelock.py:274] Lock 140539912818192 acquired on /Users/shannon/.cache/huggingface/transformers/0a80689755fc45e25c71b1b136f664c8c42622c22fa0d3846105f08182ee040f.4e79c48f578f6c9787d9a5e0b5dee9800ab9f7b5527c343de4b22139b4b950f4.lock


Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

I0928 13:51:55.200149 4404223424 filelock.py:318] Lock 140539912818192 released on /Users/shannon/.cache/huggingface/transformers/0a80689755fc45e25c71b1b136f664c8c42622c22fa0d3846105f08182ee040f.4e79c48f578f6c9787d9a5e0b5dee9800ab9f7b5527c343de4b22139b4b950f4.lock
I0928 13:51:55.311869 4404223424 filelock.py:274] Lock 140539912818192 acquired on /Users/shannon/.cache/huggingface/transformers/e652b8191d15aa872baecf86bee43ca3a9c7051ea1a0ca419a00d97cb6edc3d4.918796645b111451dabe315f42e653b50349145063ecb13ad1e7205cd75dd219.lock


Downloading:   0%|          | 0.00/459 [00:00<?, ?B/s]

I0928 13:51:55.479192 4404223424 filelock.py:318] Lock 140539912818192 released on /Users/shannon/.cache/huggingface/transformers/e652b8191d15aa872baecf86bee43ca3a9c7051ea1a0ca419a00d97cb6edc3d4.918796645b111451dabe315f42e653b50349145063ecb13ad1e7205cd75dd219.lock


In [4]:
UNICODE_CATEGORIES_TO_REPLACE = ["Cc", "Cf", "Co", "Cs", "Mn", "Zl", "Zp", "Zs"]

In [22]:
def calculate_tokenization_len(row):    
    uni_code = r'\u' + row['Character'][2:]
    s = uni_code.encode().decode('unicode_escape')
    return len(tokenizer(s, add_special_tokens=False)['input_ids'])

In [6]:
df['Tokenization Length'] = df.apply(calculate_tokenization_len, axis=1)

In [7]:
df.groupby('Code')['Tokenization Length'].agg(['count', 'mean']).loc[UNICODE_CATEGORIES_TO_REPLACE]

,count,mean
Code,,
Cc,65,0.000000
Cf,163,0.766871
Co,6,0.666667
Cs,6,0.000000
Mn,1950,0.549744
Zl,1,0.000000
Zp,1,0.000000
Zs,17,0.000000


In [8]:
df[df['Tokenization Length']==0].Code.unique()

array(['Cc', 'Cf', 'Co', 'Cs', 'Mc', 'Mn', 'So', 'Zl', 'Zp', 'Zs'],
      dtype=object)

In [9]:
df[df['Tokenization Length']==0]['Character'].to_csv("zero-length-unicode-chars.txt", index=None, header=None)

## Check on All Unicode Characters

In [11]:
import sys
import unicodedata

In [17]:
ALL_UNICODE_CHARS = [[chr(i),unicodedata.category(chr(i))] for i in range(sys.maxunicode+1)]

In [31]:
df_complete = pd.DataFrame(ALL_UNICODE_CHARS, columns=['Character', 'Code'])

In [34]:
def calculate_tokenization_len(row):    
    return len(tokenizer(row['Character'], add_special_tokens=False)['input_ids'])

In [35]:
df_complete['Tokenization Length'] = df_complete.apply(calculate_tokenization_len, axis=1)

In [37]:
len(df_complete[df_complete['Tokenization Length'] == 0]['Code'])

141318

In [45]:
df_complete.groupby('Code')['Tokenization Length'].agg(['count', 'mean']).sort_values('mean', ascending=True)

,count,mean
Code,,
Cc,65,0.000000
Zl,1,0.000000
Zp,1,0.000000
Cs,2048,0.000000
Zs,17,0.000000
Co,137468,0.000000
Cf,151,0.006623
Mn,1690,0.072781
So,5777,0.999827


In [40]:
df_complete[df_complete['Tokenization Length']==0].Code.unique()

array(['Cc', 'Zs', 'Cf', 'Mn', 'Zl', 'Zp', 'Cs', 'Co', 'So'], dtype=object)

In [46]:
df_complete[(df_complete['Tokenization Length']==0) & (df_complete['Code']=='So')]

,Character,Code,Tokenization Length
65533,�,So,0


In [50]:
df_complete[df_complete['Tokenization Length']==0]['Character'].apply(lambda e: ord(e)).to_csv("zero-length-unicode-ord.txt", index=None, header=None,  encoding='utf-8')

In [51]:
len(df_complete[df_complete['Tokenization Length']==0])

141318